# Simplex Method

Below is defined a function which comprises the Simplex Method to solve Linear Programming Problems where we have to minimize or maximize an objective function under some restrictions.

In [17]:
def simplex(A, b, c, variables, itera = 1):
    
    # Printing our initial parameters for this iteration
    print("_______________________________________________________\n")
    print("\n\n\033[1mIteration ", itera, "\033[0m")
    print("[\033[94m x_B \033[0m, \033[91m x_N \033[0m] = [ \033[94m"
          + ', '.join(variables[:len(b)]) + "\033[0m, \033[91m" + ', '.join(variables[len(b):]) + "\033[0m ]\n")
    print("Matriz A")
    print(A)
    print("\nVector b")
    print(b)
    print("\nVector c")
    print(c)
    
    # Partioning our matrix A into a basic matrix B and a not basic matrix N
    m, n = np.shape(A)
    B = A[:m, :m]
    N = A[:m, m:]
    
    # Calculating and printing our Basis, Basis Inverse and Basic Feasible Solution BFS
    B_inv = np.linalg.inv(B)
    x_b = np.array(B_inv@b)
    x_b = x_b[0, :]
    x_n = np.zeros(n - m)
    x = np.concatenate((x_b, x_n), axis = 0)
    print("_______________________________________________________\n")
    print("\nBase")
    print(B)
    print("\nBase Inversa")
    print(B_inv)
    print("\nBFS")
    print(x)
    
    # Calculating the z's minus c's and finding the maximum one with its index k
    j_index = [i for i in range(m, n)]
    c_b = c[:m]
    zs = np.zeros(len(j_index))
    z_minus_c = np.zeros(len(j_index))
    
    for i in range(len(j_index)):
        a_j = A[:,j_index[i]]
        zs[i] = c_b@B_inv@a_j
        z_minus_c[i] = zs[i] - c[j_index[i]]
        
    z_minus_c_max = max(z_minus_c)
    k = m + np.argmax(z_minus_c)
    
    # Validating if we stop or not the optimization
    if z_minus_c_max <= 0:   # We stop
        
        # Printing the final results
        print("_______________________________________________________\n")
        print("\n\n\033[1mOptimalidad alcanzada\033[0m")
        print("\nLA optima BFS es ")
        results_ls = [str(round(x_i, 4)) for x_i in x]
        print("\033[92m[ " + ', '.join(variables) + " ] = [ " + ', '.join(results_ls) + " ]\033[0m")
        perf_z = c.T@x
        print("\n z =", perf_z)
        
    else:   # We continue
        
        # Calculating and printing the y_ki's
        y_k = np.ravel(B_inv@A[:,k])
        print("_______________________________________________________\n")
        print("\nk =", k + 1, "-> column no.", k + 1, "de Matriz A (", variables[k], ")")
        print("y_k")
        print(y_k)
        
        # Analyzing if the optimal BFS is or not boundable by the condition y_k > 0
        flag = True
        aux_counter = 0
        for y_i in y_k:
            if y_i > 0:
                aux_counter = aux_counter + 1      
        if aux_counter == 0:
            flag = False

        if flag:   # Boundable
            
            # Calculating and printing x_k by the minimum quotient (current BFS divided by the y_k > 0) with its index r
            quot_ls = list()
            index_r_ls = list()
            for i in range(m):
                if y_k[i] > 0:
                    quot_ls.append(x[i] / y_k[i])
                    index_r_ls.append(i)
            quot_arr = np.array(quot_ls)
            index_r_arr = np.array(index_r_ls)
            r = index_r_arr[np.argmin(quot_arr)]
            x_Br = min(quot_arr)
            print("_______________________________________________________\n")
            print("\nr =", r + 1, "-> column no.", r + 1, "de la base (", variables[r], ")")
            print("x_Br")
            print(x_Br)

            # Exchanging the basic variable with index r and the not basic variable with index k, of the matrix A partitions
            var_aux = np.array(B[:, r])
            B[:, r] = N[:, k - m]
            N[:, k - m] = var_aux
            A = np.concatenate((B, N), axis = 1)
            
            # Exchanging the values located on indexes r and k of the vector c
            var_aux_c = c[r]
            c[r] = c[k]
            c[k] = var_aux_c
            
            # Exchanging the indexes
            print("_______________________________________________________\n")
            print("\n\033[94m", variables[k], "enters\033[0m and \033[91m", variables[r], "leaves\033[0m the basis")
            aux_index = variables[r]
            variables[r] = variables[k]
            variables[k] = aux_index
            
            # Recursive call
            simplex(A, b, c, variables, itera + 1)

        else:   # Not boundable
            print("_______________________________________________________\n")
            print("\n\n\033[1mProceso de optimización detenido :(\033[0m")
            print("\nEl BFS óptimo no es acotable")
            

In [18]:
import numpy as np
import copy

# Settings for printing numpy arrays
np.set_printoptions(precision=4, edgeitems=10)
np.core.arrayprint._line_width = 180

In [19]:
A = np.matrix([[25, 15, 25, 0, 0, -1, 0, 0],
               [25, 15, 25, 1, 0, 0, 0, 0],
               [15, 30, 20, 0, 0, 0, -1, 0],
               [5, 12, 8, 0, 0, 0, 0, -1], 
               [5, 12, 8, 0, 1, 0, 0, 0]])
b = np.array([18, 22, 20, 6, 12])
c = np.array([0.1, 0.08, 0.12, 0, 0, 0, 0, 0])
variables = ["x1", "x2", "x3", "x5", "x8", "x4", "x6", "x7"]

In [20]:
simplex(copy.deepcopy(A), copy.deepcopy(b), copy.deepcopy(c), copy.deepcopy(variables))

_______________________________________________________



Iteration  1 
[ x_B ,  x_N ] = [ x1, x2, x3, x5, x8, x4, x6, x7 ]

Matriz A
[[25 15 25  0  0 -1  0  0]
 [25 15 25  1  0  0  0  0]
 [15 30 20  0  0  0 -1  0]
 [ 5 12  8  0  0  0  0 -1]
 [ 5 12  8  0  1  0  0  0]]

Vector b
[18 22 20  6 12]

Vector c
[0.1  0.08 0.12 0.   0.   0.   0.   0.  ]
_______________________________________________________


Base
[[25 15 25  0  0]
 [25 15 25  1  0]
 [15 30 20  0  0]
 [ 5 12  8  0  0]
 [ 5 12  8  0  1]]

Base Inversa
[[-6.6613e-18  0.0000e+00  4.0000e-01 -1.0000e+00  0.0000e+00]
 [-4.4444e-02  0.0000e+00  1.6667e-01 -2.7778e-01  0.0000e+00]
 [ 6.6667e-02  0.0000e+00 -5.0000e-01  1.1667e+00  0.0000e+00]
 [-1.0000e+00  1.0000e+00 -0.0000e+00  0.0000e+00 -0.0000e+00]
 [ 0.0000e+00  0.0000e+00  0.0000e+00 -1.0000e+00  1.0000e+00]]

BFS
[ 2.      0.8667 -1.8     4.      6.      0.      0.      0.    ]
_______________________________________________________


k = 7 -> column no. 7 de Matriz A ( x